In [5]:
!pip install fastapi uvicorn[standard] python-multipart streamlit requests tensorflow opencv-python mediapipe numpy scipy pillow


In [6]:
import numpy as np
import cv2
import mediapipe as mp
from PIL import Image
import io


In [7]:
def classify_species(image_bytes: bytes) -> dict:
    try:
        image = Image.open(io.BytesIO(image_bytes))
        width, height = image.size
        if width < height:
            species = "Gir Cow"
            confidence = np.random.uniform(0.90, 0.99)
        else:
            species = "Murrah Buffalo"
            confidence = np.random.uniform(0.88, 0.98)
        print(f"✅ Classifier simulation successful: Detected {species}")
        return {"species": species, "confidence": float(confidence)}
    except Exception as e:
        print(f"❌ Error in classifier: {e}")
        return { "species": "Error", "confidence": 0.0 }


In [8]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, model_complexity=1, min_detection_confidence=0.5)

def extract_keypoints_from_video(video_path: str) -> list:
    cap = cv2.VideoCapture(video_path)
    keypoints_data = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)
        if results.pose_landmarks:
            frame_keypoints = {}
            for i, landmark in enumerate(results.pose_landmarks.landmark):
                if i in [23, 24, 25, 26, 27, 28]:
                    landmark_name = mp_pose.PoseLandmark(i).name
                    frame_keypoints[landmark_name] = {
                        "x": landmark.x,
                        "y": landmark.y,
                        "z": landmark.z,
                        "visibility": landmark.visibility
                    }
            if frame_keypoints:
                keypoints_data.append(frame_keypoints)
    cap.release()
    print(f"✅ Pose estimation successful: Extracted keypoints from {len(keypoints_data)} frames.")
    return keypoints_data


In [9]:
def calculate_mobility_score(keypoints_data: list) -> dict:
    if len(keypoints_data) < 15:
        return {"mobility_score": -1, "score_type": "Insufficient Video Length"}

    left_hip_y = [frame.get('LEFT_HIP', {}).get('y') for frame in keypoints_data]
    right_hip_y = [frame.get('RIGHT_HIP', {}).get('y') for frame in keypoints_data]
    left_hip_y = [y for y in left_hip_y if y is not None]
    right_hip_y = [y for y in right_hip_y if y is not None]

    if len(left_hip_y) < 10 or len(right_hip_y) < 10:
        return {"mobility_score": -1, "score_type": "Could Not Track Hips"}

    std_dev_left = np.std(left_hip_y)
    std_dev_right = np.std(right_hip_y)
    avg_bobbing_motion = (std_dev_left + std_dev_right) / 2

    if avg_bobbing_motion < 0.01:
        score = 0
        score_type = "Prime Mobility"
    elif avg_bobbing_motion < 0.025:
        score = 1
        score_type = "At-Risk"
    else:
        score = 2
        score_type = "Lame Mobility Detected"

    print(f"✅ Gait analysis successful: Score {score}, Motion value {avg_bobbing_motion:.4f}")
    return {"mobility_score": score, "score_type": score_type}


In [10]:
# Replace this path with your cow/buffalo video file
video_path = "sample_video.mp4"

# Step 1: classify using first frame
cap = cv2.VideoCapture(video_path)
ret, frame = cap.read()
cap.release()
if ret:
    _, buffer = cv2.imencode('.jpg', frame)
    first_frame_bytes = buffer.tobytes()
    classification_result = classify_species(first_frame_bytes)
else:
    classification_result = {"species": "Error", "confidence": 0.0}

# Step 2: extract keypoints
keypoints_data = extract_keypoints_from_video(video_path)

# Step 3: gait analysis
gait_result = calculate_mobility_score(keypoints_data)

# Final Results
print("\n=== FINAL RESULTS ===")
print("Classification:", classification_result)
print("Gait Analysis:", gait_result)


✅ Pose estimation successful: Extracted keypoints from 0 frames.

=== FINAL RESULTS ===
Classification: {'species': 'Error', 'confidence': 0.0}
Gait Analysis: {'mobility_score': -1, 'score_type': 'Insufficient Video Length'}
